# Install and import required libraries

In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import google.colab
from tqdm import tqdm
import re
import warnings
import csv

from flair.models import SequenceTagger
from collections import Counter
from flair.datasets import UD_INDONESIAN
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.data import Sentence

warnings.filterwarnings("ignore")

# Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df_tweets = pd.read_csv('/content/drive/MyDrive/TA/tweets/ruhutsitompul/done_labelled_ruhutsitompul.csv', delimiter=',', error_bad_lines=False)
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,...,clean_tweets_len,Month,social,historical,dehumanization,accusation,attack,loyalty,threat,Tweets.1
0,0,1585739644011892736,2022-10-27 21:06:07,1585739644011892736,460,80,Sama banget Komentar ini dgn para pendukung se...,Sama banget Komentar ini dgn para pendukung se...,NaN,NaN,...,11,October,1,0,0,0,0.0,0,0,Sama banget Komentar ini dgn para pendukung se...
1,1,1585737538806853632,2022-10-27 20:57:45,1585737538806853632,420,92,Ha ha ha sigundul penguasa ancol karena selama...,Ha ha ha sigundul penguasa ancol karena selama...,NaN,NaN,...,15,October,1,0,0,1,0.0,0,0,Ha ha ha sigundul penguasa ancol karena selama...
2,2,1584780616369049605,2022-10-25 05:35:17,1584780616369049605,343,40,Kasihan Ibu ini jadi korban akibat dicuci otak...,Kasihan Ibu ini jadi korban akibat dicuci otak...,NaN,NaN,...,14,October,0,0,1,1,0.0,0,0,Kasihan Ibu ini jadi korban akibat dicuci otak...
3,3,1584778048120836096,2022-10-25 05:25:05,1584778048120836096,212,23,Ha ha hakadrun pada stresssssss mengenai beber...,Ha ha hakadrun pada stresssssss mengenai beber...,NaN,NaN,...,18,October,1,0,1,0,0.0,0,0,Ha ha hakadrun pada stresssssss mengenai beber...
4,4,1584774254502912000,2022-10-25 05:10:00,1584774254502912000,860,63,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,NaN,NaN,...,33,October,0,0,1,0,0.0,0,0,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4178,4178,1118666888647602177,2019-04-18 00:05:44,1118666888647602177,0,0,@NediSetiadi Kami sedih krn biar sudah kalian ...,@NediSetiadi Kami sedih krn biar sudah kalian ...,NaN,NaN,...,14,April,1,0,0,1,0.0,0,0,@NediSetiadi Kami sedih krn biar sudah kalian ...
4179,4179,1118665503642247170,2019-04-18 00:00:14,1118665503642247170,7,0,@TanYoana haiya Amoy ini makin stresssssss kac...,@TanYoana haiya Amoy ini makin stresssssss kac...,NaN,NaN,...,5,April,0,0,0,0,0.0,0,0,@TanYoana haiya Amoy ini makin stresssssss kac...
4180,4180,1118664923301572608,2019-04-17 23:57:56,1118664923301572608,1684,200,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...,NaN,NaN,...,31,April,0,0,0,0,0.0,0,0,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...
4181,4181,1118396091148460032,2019-04-17 06:09:41,1118396091148460032,6,0,"@panca66 Ha ha ha maaf memilih diri sendiri, s...","@panca66 Ha ha ha maaf memilih diri sendiri, s...",NaN,NaN,...,5,April,0,0,0,0,0.0,0,0,"@panca66 Ha ha ha maaf memilih diri sendiri, s..."


In [ ]:
# Remove tweets without any aspect
df_tweets = df_tweets.loc[(df_tweets[['social', 'historical', 'dehumanization', 'accusation', 'attack', 'loyalty', 'threat']] != 0).any(axis=1)]
df_tweets = df_tweets.reset_index(drop=True)
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,...,clean_tweets_len,Month,social,historical,dehumanization,accusation,attack,loyalty,threat,Tweets.1
0,0,1585739644011892736,2022-10-27 21:06:07,1585739644011892736,460,80,Sama banget Komentar ini dgn para pendukung se...,Sama banget Komentar ini dgn para pendukung se...,NaN,NaN,...,11,October,1,0,0,0,0.0,0,0,Sama banget Komentar ini dgn para pendukung se...
1,1,1585737538806853632,2022-10-27 20:57:45,1585737538806853632,420,92,Ha ha ha sigundul penguasa ancol karena selama...,Ha ha ha sigundul penguasa ancol karena selama...,NaN,NaN,...,15,October,1,0,0,1,0.0,0,0,Ha ha ha sigundul penguasa ancol karena selama...
2,2,1584780616369049605,2022-10-25 05:35:17,1584780616369049605,343,40,Kasihan Ibu ini jadi korban akibat dicuci otak...,Kasihan Ibu ini jadi korban akibat dicuci otak...,NaN,NaN,...,14,October,0,0,1,1,0.0,0,0,Kasihan Ibu ini jadi korban akibat dicuci otak...
3,3,1584778048120836096,2022-10-25 05:25:05,1584778048120836096,212,23,Ha ha hakadrun pada stresssssss mengenai beber...,Ha ha hakadrun pada stresssssss mengenai beber...,NaN,NaN,...,18,October,1,0,1,0,0.0,0,0,Ha ha hakadrun pada stresssssss mengenai beber...
4,4,1584774254502912000,2022-10-25 05:10:00,1584774254502912000,860,63,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,NaN,NaN,...,33,October,0,0,1,0,0.0,0,0,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,4151,1120875995962101760,2019-04-24 02:23:56,1120875995962101760,26,4,"@ustadtengkuzul Belajarlah menerima kekalahan,...","@ustadtengkuzul Belajarlah menerima kekalahan,...",NaN,NaN,...,12,April,1,0,0,1,0.0,0,0,"@ustadtengkuzul Belajarlah menerima kekalahan,..."
1660,4153,1120574450410217472,2019-04-23 06:25:42,1120574450410217472,1660,264,Aneh tapi Nyata ini bukan HOAX di Uk...,Aneh tapi Nyata ini bukan HOAX di Uk...,NaN,NaN,...,13,April,0,0,0,1,0.0,0,0,Aneh tapi Nyata ini bukan HOAX di Uk...
1661,4166,1119465028040527872,2019-04-20 04:57:15,1119465028040527872,1311,193,"Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr...","Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr...",NaN,NaN,...,24,April,1,0,0,1,0.0,0,0,"Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr..."
1662,4168,1119429279811428353,2019-04-20 02:35:12,1119429279811428353,10,0,@FerdinandHutah2 Masih saja kau menggonggong s...,@FerdinandHutah2 Masih saja kau menggonggong s...,NaN,NaN,...,6,April,0,0,1,0,0.0,0,0,@FerdinandHutah2 Masih saja kau menggonggong s...


# POS tagging with flair

In [ ]:
# 1. get the corpus
corpus = UD_INDONESIAN().downsample(0.1)
print(corpus)

# 2. what label do we want to predict?
label_type = 'upos'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

# 4. initialize embeddings
embedding_types = [

    WordEmbeddings('id-crawl'),
    WordEmbeddings('id'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-upos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

2023-03-13 07:49:11,603 Reading data from /root/.flair/datasets/ud_indonesian
2023-03-13 07:49:11,605 Train: /root/.flair/datasets/ud_indonesian/id_gsd-ud-train.conllu
2023-03-13 07:49:11,609 Dev: /root/.flair/datasets/ud_indonesian/id_gsd-ud-dev.conllu
2023-03-13 07:49:11,614 Test: /root/.flair/datasets/ud_indonesian/id_gsd-ud-test.conllu
2023-03-13 07:49:22,075 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-03-13 07:49:22,833 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-03-13 07:49:26,055 Warning: An empty Sentence was created! Are there empty strings in your dataset?
Corpus: 448 train + 56 dev + 56 test sentences
2023-03-13 07:49:26,067 Computing label dictionary. Progress:


448it [00:00, 9409.68it/s]

2023-03-13 07:49:26,167 Dictionary created for label 'upos' with 17 values: NOUN (seen 2096 times), PROPN (seen 1778 times), PUNCT (seen 1410 times), VERB (seen 1010 times), ADP (seen 919 times), PRON (seen 507 times), ADJ (seen 398 times), NUM (seen 342 times), DET (seen 313 times), ADV (seen 306 times), CCONJ (seen 272 times), SCONJ (seen 168 times), AUX (seen 165 times), PART (seen 91 times), SYM (seen 12 times), X (seen 10 times)
Dictionary with 17 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, ADV, CCONJ, SCONJ, AUX, PART, SYM, X


2023-03-13 07:49:52,088 SequenceTagger predicts: Dictionary with 17 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, ADV, CCONJ, SCONJ, AUX, PART, SYM, X
2023-03-13 07:49:52,151 ----------------------------------------------------------------------------------------------------
2023-03-13 07:49:52,154 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'id-crawl'
      (embedding): Embedding(1000000, 300)
    )
    (list_embedding_1): WordEmbeddings(
      'id'
      (embedding): Embedding(300686, 300)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=600, out_features=600, bias=True)
  (rnn): LSTM(600, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=19, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2023-03-13 07:49:52,158 --------------------------------------------------------------

100%|██████████| 2/2 [00:00<00:00,  2.57it/s]

...

2023-03-13 07:53:32,413 ----------------------------------------------------------------------------------------------------
2023-03-13 07:53:36,811 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, ADV, CCONJ, SCONJ, AUX, PART, SYM, X, <START>, <STOP>


100%|██████████| 2/2 [00:00<00:00,  3.90it/s]

2023-03-13 07:53:38,079 Evaluating as a multi-label problem: False
2023-03-13 07:53:38,104 0.8153	0.8153	0.8153	0.8153
2023-03-13 07:53:38,106 
Results:
- F-score (micro) 0.8153
- F-score (macro) 0.6294
- Accuracy 0.8153

By class:
              precision    recall  f1-score   support

       PROPN     0.7810    0.9772    0.8682       219
        NOUN     0.7956    0.7682    0.7817       233
       PUNCT     1.0000    1.0000    1.0000       178
         ADP     0.9259    0.8850    0.9050       113
        VERB     0.8868    0.8468    0.8664       111
        PRON     0.7222    0.9123    0.8062        57
         ADV     0.2857    0.3243    0.3038        37
         ADJ     0.6087    0.2917    0.3944        48
       CCONJ     0.8750    0.8235    0.8485        34
         DET     0.8000    0.5333    0.6400        30
         NUM     0.8636    0.7600    0.8085        25
         AUX     0.7500    0.3000    0.4286        20
       SCONJ     0.2500    0.1333    0.1739        15
        PAR

2023-03-13 07:53:38,109 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8152753108348135,
 'dev_score_history': [0.18403755868544602,
  0.4244131455399061,
  0.507981220657277,
  0.6093896713615023,
  0.6497652582159624,
  0.6995305164319249,
  0.7051643192488263,
  0.7474178403755869,
  0.7718309859154929,
  0.7990610328638498],
 'train_loss_history': [2.686312702050851,
  2.074105638404933,
  1.7478469797235443,
  1.505616168872646,
  1.3263712594177426,
  1.188179990327077,
  1.0733475988344647,
  0.9719774807498469,
  0.9183129441729612,
  0.845368774202243],
 'dev_loss_history': [2.2897210121154785,
  1.7922370433807373,
  1.5038204193115234,
  1.2395789623260498,
  1.1096489429473877,
  0.9337056279182434,
  0.84358149766922,
  0.7563403248786926,
  0.6895579099655151,
  0.6210797429084778]}

#3-gram

In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# Tokenize and filter tweets with category 1
tweets = df_tweets[df_tweets["attack"] == 1]["clean_tweets"].tolist()

# Create trigrams
trigrams = []
for tweet in tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s.tokens]
    trigrams.extend(zip(tags, tags[1:], tags[2:]))

# Count trigrams and get top 3 most frequent ones
trigram_counts = Counter(trigrams)
top_n_trigrams = trigram_counts.most_common(10)

# Print top trigrams
for trigram, count in top_n_trigrams:
    print("Trigram: {} - Count: {}".format(trigram, count))

2023-03-13 07:53:42,083 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, ADV, CCONJ, SCONJ, AUX, PART, SYM, X, <START>, <STOP>
Trigram: ('PROPN', 'PROPN', 'PROPN') - Count: 72
Trigram: ('NOUN', 'NOUN', 'NOUN') - Count: 64
Trigram: ('NOUN', 'NOUN', 'ADJ') - Count: 23
Trigram: ('VERB', 'NOUN', 'NOUN') - Count: 19
Trigram: ('NOUN', 'ADJ', 'NOUN') - Count: 15
Trigram: ('PROPN', 'VERB', 'NOUN') - Count: 14
Trigram: ('NOUN', 'PROPN', 'PROPN') - Count: 11
Trigram: ('PROPN', 'PROPN', 'VERB') - Count: 10
Trigram: ('ADJ', 'NOUN', 'NOUN') - Count: 8
Trigram: ('NOUN', 'NOUN', 'PROPN') - Count: 7


In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# Tokenize and filter tweets with category 1
tweets = df_tweets[df_tweets["loyalty"] == 1]["clean_tweets"].tolist()

# Create trigrams
trigrams = []
for tweet in tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s.tokens]
    trigrams.extend(zip(tags, tags[1:], tags[2:]))

# Count trigrams and get top 3 most frequent ones
trigram_counts = Counter(trigrams)
top_n_trigrams = trigram_counts.most_common(10)

# Print top trigrams
for trigram, count in top_n_trigrams:
    print("Trigram: {} - Count: {}".format(trigram, count))

2023-03-13 07:53:45,836 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, ADV, CCONJ, SCONJ, AUX, PART, SYM, X, <START>, <STOP>
Trigram: ('PROPN', 'PROPN', 'PROPN') - Count: 176
Trigram: ('NOUN', 'NOUN', 'NOUN') - Count: 33
Trigram: ('NOUN', 'PROPN', 'PROPN') - Count: 18
Trigram: ('PROPN', 'PROPN', 'VERB') - Count: 14
Trigram: ('NOUN', 'NOUN', 'ADJ') - Count: 13
Trigram: ('PROPN', 'VERB', 'NOUN') - Count: 13
Trigram: ('VERB', 'NOUN', 'PROPN') - Count: 12
Trigram: ('PROPN', 'NOUN', 'NOUN') - Count: 9
Trigram: ('PROPN', 'PROPN', 'AUX') - Count: 9
Trigram: ('PROPN', 'PROPN', 'NOUN') - Count: 8


In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# Tokenize and filter tweets with category 1
tweets = df_tweets[df_tweets["threat"] == 1]["clean_tweets"].tolist()

# Create trigrams
trigrams = []
for tweet in tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s.tokens]
    trigrams.extend(zip(tags, tags[1:], tags[2:]))

# Count trigrams and get top 3 most frequent ones
trigram_counts = Counter(trigrams)
top_n_trigrams = trigram_counts.most_common(10)

# Print top trigrams
for trigram, count in top_n_trigrams:
    print("Trigram: {} - Count: {}".format(trigram, count))

2023-03-13 07:53:49,739 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, ADJ, NUM, DET, ADV, CCONJ, SCONJ, AUX, PART, SYM, X, <START>, <STOP>
Trigram: ('PROPN', 'PROPN', 'PROPN') - Count: 283
Trigram: ('NOUN', 'NOUN', 'NOUN') - Count: 87
Trigram: ('NOUN', 'PROPN', 'PROPN') - Count: 37
Trigram: ('NOUN', 'NOUN', 'ADJ') - Count: 37
Trigram: ('PROPN', 'PROPN', 'VERB') - Count: 28
Trigram: ('PROPN', 'VERB', 'NOUN') - Count: 28
Trigram: ('NOUN', 'NOUN', 'PROPN') - Count: 28
Trigram: ('PROPN', 'NOUN', 'NOUN') - Count: 25
Trigram: ('VERB', 'NOUN', 'NOUN') - Count: 22
Trigram: ('PROPN', 'PROPN', 'NOUN') - Count: 20
